### Workflow

1. **DONE** - Split the dataset into a training and a testing subset. Use the category “title” for the testing set and the categories “comment” and “post” for the training set. The short length of titles will make them good candidates later as seeds for text generation.
1. **DONE** - Build the matrix of prefix—word frequencies.
  + Use the `ngrams` function from `nltk.utils` to generate all n-grams from the corpus
  + Set the following `left_pad_symbol = <s> and right_pad_symbol = </s>`
1. **DONE** - Write a text generation function:
  + takes a bigram as input and generates the next token
  + iteratively slide the prefix over the generated text so that the new prefix includes the most recent token; generates the next token
  + to generate each next token, sample the list of words associated with the prefix using the probability distribution of the prefix
  + stop the text generation when a certain number of words have been generated or the latest token is a `</s>`.
1. **DONE** - Write a function that can estimate the probability of a sentence and use it to select the most probable sentence out of several candidate sentences.
 + Split the sentence into trigrams and use the chain rule to calculate the probability of the sentence as a product of the bigrams—tokens probabilities
1. **IN PROGRESS** - Implement the **perplexity** scoring function for a given sentence and for the training corpus.
  + The perplexity of a corpus is obtained by multiplying the perplexity of each sentence in the corpus.
1. **DONE** - Implement **Additive Laplace** smoothing to give a non-zero probability to missing prefix—token combinations when calculating perplexity.
1. Calculate the perplexity of the language model on the test set composed of titles.
1. Try to improve the perplexity score of your model by:
  + modifying the preprocessing phase of the corpus,
  + increasing or decreasing number of tokens in the model (bi grams, 4-grams, etc.),
  + varying the delta parameter in the Additive Laplace smoothing step.

#### Laplace Smoothing 

$$ p(w_{1} w_2 w_3) = p(w_3 / w_{1} w_{2}) = \frac{count(w_{1}w_{2} w_3)+ \delta} {count(w_{1}w_{2}) + \delta \times |N|}$$

$$p(token / prefix) = \frac{count( prefix + token) + \delta} {count(prefix) + \delta \times | N |}$$

##### n-gram is missing from the corpus

$$p(token / prefix) = \frac{ \delta} {count(prefix) + \delta \times | N |}$$

##### prefix is missing from corpus

$$p(token / prefix) = \frac{ \delta} { \delta \times | N |}$$

* $ | N | $ is the size of size of the vocabulary
* $ \delta $ is a constant that's less than 1

### Resources

* https://programminghistorian.org/en/lessons/counting-frequencies
* https://www.kite.com/python/docs/nltk.ngrams
* https://stackoverflow.com/questions/54941966/how-can-i-calculate-perplexity-using-nltk (using off the shelf NLTK for perplexity and MLE bigram estimates, etc.)
* https://stats.stackexchange.com/questions/129352/how-to-find-the-perplexity-of-a-corpus 

In [2]:
import pandas as pd

In [3]:
# Loading the data
TOK_FILE= 'stackexchange_tokenized.csv'
df = pd.read_csv(f'data/{TOK_FILE}')

In [4]:
df['category'].unique()

array(['title', 'post', 'comment'], dtype=object)

#### Split into Training and Test Set

In [5]:
title = df['category'] == 'title'
posts = df['category'] == 'post'
commt = df['category'] == 'comment'

test_df  = df[title]
train_df = df[posts | commt] 

In [6]:
test_df.count()

Unnamed: 0    91648
post_id       91648
parent_id         0
comment_id        0
text          91648
category      91648
length        91648
tokenized     91648
dtype: int64

In [7]:
train_df.count()

Unnamed: 0    717518
post_id       717518
parent_id      75430
comment_id    550329
text          717518
category      717518
length        717518
tokenized     717518
dtype: int64

#### Build the matrix of prefix—word frequencies

In [8]:
from nltk.util import ngrams
from collections import Counter, defaultdict
import numpy as np
import math




In [9]:
raw_counts = defaultdict(Counter) 
freq = dict()
vocab = set()

In [10]:

rng = np.random.default_rng()

In [22]:
def bigram_counts(s_string):
    global vocab, raw_counts
    s_list = s_string.split(' ')
    vocab.update({*s_list})
    
    for tg in ngrams(s_list, 3, pad_left=True, pad_right=True, right_pad_symbol='</s>', left_pad_symbol='<s>'):
        bg = tg[:-1]
        wd = tg[-1]
        if not bg in raw_counts:
            raw_counts[bg] = Counter()
        raw_counts[bg][wd] += 1


def counts_to_freq():
    for bg, cntr in raw_counts.items():
        tot = sum(cntr.values())
        freq[bg] = { i:cntr[i]/tot for i in cntr }
        
def sample_next(bg):
    nwd = freq[tuple(bg)]
    nw  = rng.choice(list(nwd.keys()), 1, p=list(nwd.values()))[0]
    return nw

#pass wc=0 if you just want to generate one sentence
def generate_text(seed, wc=0):
    
    if type(seed) == str:
        seed = ['<s>', 'I'] + seed.split(' ') #won't matter if there are extra things on the list
    bg = seed[-2:]
    
    gt = bg #this could be cleaned up
    
    one_sentence = False
    if wc==0:
        one_sentence = True
        wc = 50
    
    while wc > 0:
        nw = sample_next(bg)
        gt.append(nw)
        wc -=1
        if one_sentence and gt[-1] == '</s>':
            break
        bg = gt[-2:]
    return gt


def prob_of_sent(s):
    probs = get_probs(s)
    
    lprobs = [math.log(p) for p in probs]
    return math.exp(sum(lprobs))

def lps_prob_of_sent(s):
    probs = laplace_smoothed_prob(s)
    
    lprobs = [math.log(p) for p in probs]
    return math.exp(sum(lprobs))


def laplace_smoothed_prob(s):
    global vocab
    probs = []
    vocab_size = len(vocab)
    delta  = 0.25  

    if type(s) == str:
        s = s.split(' ') #small error. Watch out passing in sentence with punctuation not space-separated from the last word.  
    
    tgrams = ngrams(s, 3,  pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>')
    #LEFT OFF HERE. NEED TO TEST THIS.
    for tg in tgrams:
        bg = tg[:-1]
        wd = tg[-1] 
        tgc = delta
        bgc = delta * vocab_size
        if bg in raw_counts:
            bgc += sum(raw_counts[bg].values())
            tgc += raw_counts[bg][wd]
        probs.append(tgc/bgc)
    return probs
                

def get_probs(s):
    if type(s) == str:
        s = s.split(' ') #small error. Watch out passing in sentence with punctuation not space-separated from the last word.  
    
    mprob  = 0.005 #TODO: what is the right value?
    probs  = []
    tgrams = ngrams(s, 3,  pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>')
    for tg in tgrams:
        bg = tg[:-1]
        wd = tg[-1]
        if not bg in freq:
            probs.append(mprob)
        else:
            pb = freq[bg][wd] if wd in freq[bg] else mprob
            probs.append(pb)
    return probs

def print_it(str):
    print(str+"\n")

def perplexity(s):
    #steps
    # * get probs for all trigrams.
    # * calc 1/p for each
    # * take the log of each 
    # * sum up the logs  
    # * multiple the result by 1/(len(s)) 
    # * exp() the result. 
    lip = [math.log(1/p) for p in get_probs(s)]
    N = len(lip)
    return math.exp((1/N) * sum(lip))



In [12]:
%%time
train_df['tokenized'].apply(bigram_counts)
print('done')

#train_df['tokenized'].head(n=150).apply(bigram_counts)

done
CPU times: user 1min 57s, sys: 937 ms, total: 1min 58s
Wall time: 1min 58s


In [13]:
%%time
counts_to_freq()
print('done')

done
CPU times: user 8.33 s, sys: 480 ms, total: 8.81 s
Wall time: 8.81 s


In [ ]:
# LEFT OFF HERE - Overflow.  
#corpus perplexity
c_perp = 0
def corpus_perplexity_step1(s):
    global c_perp
    c_perp = c_perp + math.log(perplexity(s))

    
# maybe helpful with overflow error 
# https://stats.stackexchange.com/questions/129352/how-to-find-the-perplexity-of-a-corpus
df['tokenized'].apply(corpus_perplexity_step1)
c_perp = math.exp(c_perp)

#### Testing

In [26]:
p = lps_prob_of_sent("Thank you") 
print(p)
p = lps_prob_of_sent("Thank you for the answer .")
print(p)
p = lps_prob_of_sent("Thank you for the butterfly .")
print(p)
p = lps_prob_of_sent("zzz xx ff afe")
print(p)

4.60285946983705e-08
6.631070049074365e-12
3.8911442646285767e-23
1.484185689441057e-35


In [33]:
p = perplexity("Thank you")
print(p)
p = perplexity("Thank you for the butterfly .")
print(p)

6.914399620199843
10.12746589825184


In [34]:
#after calculating this by hand it's correct. 
lip = [math.log(1/p) for p in [1/2, 1/3, 1/10]]
N = len(lip)
math.exp((1/N) * sum(lip))

3.9148676411688643

In [19]:
list(ngrams("Thank you".split(" "), 3,  pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>'))

[('<s>', '<s>', 'Thank'),
 ('<s>', 'Thank', 'you'),
 ('Thank', 'you', '</s>'),
 ('you', '</s>', '</s>')]

In [23]:
start_counter = raw_counts[('<s>', '<s>')]
print(start_counter['Thank'])
print(start_counter['I'])

12393
120395


In [29]:
p = prob_of_sent("Thank you") #surprised this isn't a higher prob. 
print(p)
p = prob_of_sent("Thank you for the answer .")
print(p)
p = prob_of_sent("Thank you for the butterfly .")
print(p)
p = prob_of_sent("zzz xx ff afe")
print(p)
p = prob_of_sent("zzz xx ff afe you .")
print(p)


0.0004375040679188112
1.21292662159124e-06
9.036363322889784e-09
1.5625000000000006e-14
5.6052210670627125e-15


In [154]:
rng.choice([',', 'you', 'no', '.'], 4, p=[0.2, 0.2, 0.2, 0.4])

array(['.', '.', 'you', '.'], dtype='<U3')

In [155]:
rng.choice([',', 'you', 'no', '.'], 1, p=[0.2, 0.2, 0.2, 0.4])[0]

'.'

In [156]:
freq[tuple(['Hello', 'there'])]

{',': 0.2, 'you': 0.2, 'no': 0.2, '.': 0.4}

In [39]:
generate_text("Hello there")

['Hello',
 'there',
 '.',
 'The',
 'unbiased',
 'maximum',
 'likelihood',
 'to',
 'produce',
 'simulated',
 'versions',
 'of',
 'the',
 'way',
 'things',
 'are',
 'not',
 'clear',
 'which',
 'models',
 'did',
 '?',
 '</s>']

In [117]:
keys = list(freq.keys())[:10]
type(keys[0])

tuple

In [116]:
list(freq.keys())[:10]

[('How', 'should'),
 ('should', 'I'),
 ('I', 'elicit'),
 ('elicit', 'prior'),
 ('prior', 'distributions'),
 ('distributions', 'from'),
 ('from', 'experts'),
 ('experts', 'when'),
 ('when', 'fitting'),
 ('fitting', 'a')]

In [106]:
list(freq.values())[:10]

[{'I': 0.7388748950461796,
  'economists': 0.0008396305625524769,
  'you': 0.012594458438287154,
  'one': 0.04953820319059614,
  'we': 0.033585222502099076,
  'such': 0.0041981528127623844,
  'meta': 0.0008396305625524769,
  'these': 0.007556675062972292,
  'it': 0.014273719563392108,
  'repeated': 0.0008396305625524769,
  'the': 0.036943744752308987,
  '/': 0.0025188916876574307,
  'decision': 0.0016792611251049538,
  'choose': 0.0008396305625524769,
  'i': 0.0218303946263644,
  'that': 0.005877413937867338,
  'she': 0.0008396305625524769,
  'a': 0.0033585222502099076,
  'criterion': 0.0008396305625524769,
  'this': 0.020990764063811923,
  'my': 0.0025188916876574307,
  "'": 0.0008396305625524769,
  'proceed': 0.0008396305625524769,
  'be': 0.0041981528127623844,
  'x4': 0.0008396305625524769,
  'tiny': 0.0025188916876574307,
  'approach': 0.0016792611251049538,
  'outliers': 0.005037783375314861,
  ',': 0.0008396305625524769,
  'they': 0.0025188916876574307,
  'data': 0.0008396305625

In [50]:
train_df.head()

,Unnamed: 0,post_id,parent_id,comment_id,text,category,length,tokenized
91648,91752,1,NaN,NaN,How should I elicit prior distributions from e...,post,84,How should I elicit prior distributions from e...
91649,91753,2,NaN,NaN,In many different statistical methods there is...,post,138,In many different statistical methods there is...
91650,91754,3,NaN,NaN,What are some valuable Statistical Analysis op...,post,191,What are some valuable Statistical Analysis op...
91651,91755,4,NaN,NaN,I have two groups of data. Each with a differe...,post,477,I have two groups of data . Each with a differ...
91652,91756,5,3.0,NaN,The R-project R is valuable and significant be...,post,289,The R - project R is valuable and significant ...


In [104]:
list(raw_counts.keys())[:10]

[('How', 'should'),
 ('should', 'I'),
 ('I', 'elicit'),
 ('elicit', 'prior'),
 ('prior', 'distributions'),
 ('distributions', 'from'),
 ('from', 'experts'),
 ('experts', 'when'),
 ('when', 'fitting'),
 ('fitting', 'a')]

In [10]:
#I need to see some counts over 1 or I'm going to expect that I have this wrong. 
list(raw_counts.values())[:10]

[Counter({'I': 880,
          'economists': 1,
          'you': 15,
          'one': 59,
          'we': 40,
          'such': 5,
          'meta': 1,
          'these': 9,
          'it': 17,
          'repeated': 1,
          'the': 44,
          '/': 3,
          'decision': 2,
          'choose': 1,
          'i': 26,
          'that': 7,
          'she': 1,
          'a': 4,
          'criterion': 1,
          'this': 25,
          'my': 3,
          "'": 1,
          'proceed': 1,
          'be': 5,
          'x4': 1,
          'tiny': 3,
          'approach': 2,
          'outliers': 6,
          ',': 1,
          'they': 3,
          'data': 1,
          'accuracy': 1,
          'LSTM': 1,
          '(': 1,
          'do': 1,
          'he': 2,
          'people': 1,
          'regression': 1,
          'interpret': 1,
          'change': 1,
          'Bonferroni': 1,
          'our': 1,
          'an': 1,
          'explain': 1,
          'then': 1,
          'imputation': 1,


In [65]:
#This shows that my bigram counter is broken. 

#seems like something I'd find.
bg  = ('I', 'know')
jbg = ' '.join(bg)
print(raw_counts[bg])

#check in the training set
found = train_df['tokenized'].str.contains(jbg)
train_df[found]['tokenized']

Counter({'this': 1})


91649     In many different statistical methods there is...
91662     Two projects spring to mind : Bugs - taking ( ...
91724     I had a plan of learning R in the near future ...
91748     Let us say a man rolls a six sided die and it ...
91779     This is one I 've used successfully : I just s...
                                ...                        
808933    I do n't know * anything * about your data oth...
808983    Thank you for the answer . yes your understand...
809075    @whuber I know how to compute p value from sta...
809076    @ŁukaszGrad I know S1 ^ 2&S2 ^ 2 shown in abov...
809081    @whuber Now I understand your previous comment...
Name: tokenized, Length: 14388, dtype: object

In [25]:
# does ngram splitting include the padding? 
i = 0
s = train_df.iloc[i]['tokenized']
print(s)

How should I elicit prior distributions from experts when fitting a Bayesian model ?


In [32]:
#Note that it starts with two padding characters. 
x = list(ngrams(s.split(' '), 3,  pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>'))
print(x)

[('<s>', '<s>', 'How'), ('<s>', 'How', 'should'), ('How', 'should', 'I'), ('should', 'I', 'elicit'), ('I', 'elicit', 'prior'), ('elicit', 'prior', 'distributions'), ('prior', 'distributions', 'from'), ('distributions', 'from', 'experts'), ('from', 'experts', 'when'), ('experts', 'when', 'fitting'), ('when', 'fitting', 'a'), ('fitting', 'a', 'Bayesian'), ('a', 'Bayesian', 'model'), ('Bayesian', 'model', '?'), ('model', '?', '</s>'), ('?', '</s>', '</s>')]


In [33]:
x = list(ngrams(s.split(' '), 3,  pad_left=False, pad_right=False, left_pad_symbol='<s>', right_pad_symbol='</s>'))
print(x)

[('How', 'should', 'I'), ('should', 'I', 'elicit'), ('I', 'elicit', 'prior'), ('elicit', 'prior', 'distributions'), ('prior', 'distributions', 'from'), ('distributions', 'from', 'experts'), ('from', 'experts', 'when'), ('experts', 'when', 'fitting'), ('when', 'fitting', 'a'), ('fitting', 'a', 'Bayesian'), ('a', 'Bayesian', 'model'), ('Bayesian', 'model', '?')]
